In [1]:
import csv
import os
import pandas as pd
from django.contrib.gis.geos import Point
from batid.services.guess_bdg_new import Guesser, ClosestFromPointHandler, GeocodeAddressHandler, GeocodeNameHandler
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

source_path = "./BATI_CD64-reencoded.csv"
work_file = "./guess.json"

In [2]:
def row_to_input(row):
    
    address_infos = [
            row["NUMERO_ADRESSE"], 
            row["ADRESSE"], 
            row["COMPLEMENT_ADRESSE"], 
            row["CODE_POSTAL"], 
            row["COMMUNE"]
        ]
    
    point = Point(float(row['COORD_X_L93']), float(row['COORD_Y_L93']), srid=2154)
    point.transform(4326)
    
    return {
        "ext_id": row["ID_BATI"],
        "address": ' '.join(address_infos),
        "lat": point[1],
        "lng": point[0]
        
        
    }

In [3]:
inputs = []

with open(source_path, 'r') as f:
    reader = csv.DictReader(f)
    
    
    for row in reader:

        inputs.append(row_to_input(row))
    
        
guesser = Guesser()
guesser.create_work_file(inputs, work_file)


    

- saving work file
- work file saved


In [4]:
guesser = Guesser()
guesser.handlers = [
    ClosestFromPointHandler(),
    GeocodeAddressHandler()
]
guesser.guess_work_file(work_file)

- loading work file
- converting guesses to batches
- converted 287 guesses to 1 batches
Batch 1/1
Batch changed
- saving work file
- work file saved


In [5]:
guesser.report()

-- Report --
Number of rows: 287
Number of match: 272 (94.77%)

-- finished_steps --
Rows with finished_steps closest_from_point: 287 (100.00%)
Rows with finished_steps geocode_address: 287 (100.00%)
Rows with empty finished_steps: 0 (0.00%)

-- match_reasons : absolute --
match_reason
point_on_bdg             257
precise_address_match      8
isolated_closest_bdg       7
Name: count, dtype: int64

-- match_reasons : % --
match_reason
point_on_bdg             89.547038
precise_address_match     2.787456
isolated_closest_bdg      2.439024
Name: count, dtype: float64

-- Inputs --
rows with ban_id: 12 (4.18%)


In [6]:
guesser.to_csv("dpt64-rnb.csv")